In [15]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import math

In [16]:
class MedianPolish:
    """Fits an additive model using Tukey's median polish algorithm"""

    #note that the self argument is default that allows reference to an instance of the
    #class that has been defined by the user
    #no argument is actally specified to be passed to the self argument by the user;
    #the first user specified argument
    #goes into the first argument listed after self in the code of the class
    def __init__(self, array): 
        """Get numeric data from numpy ndarray to self.tbl, keep the original copy in tbl_org"""
        #checks if the argument called array is actually a numerical array of the
        #object type np.ndarray by using the comparison function isinstance
        if isinstance(array, np.ndarray):
            self.tbl_org = array
            #tbl_org will allow you to compare the final polished matrix with row and column
            #effects removed to the original unpolished matrix
            self.tbl = self.tbl_org.copy()
        else:
            raise TypeError('Expected the argument to be a numpy.ndarray.')

    @staticmethod
    def csv_to_ndarray(fname): 
        """ Utility method for loading ndarray from .csv file""" 
        try:
            #generates an array-like object of type np.ndarray
            #from a comma separated values file
            return np.genfromtxt(fname, delimiter=",")	
        except Exception, e:
            print "Error loading file %s:" % fname
            raise

    def median_polish(self, max_iterations, method):
        """
            Implements Tukey's median polish alghoritm for additive models
            method - default is median, alternative is mean. That would give us result equal ANOVA.
        """
        
        grand_effect = 0
        median_row_effects = 0
        median_col_effects = 0
        #defines a vector that stores the row_effects during each iteration;
        #the vector is initialized with zeros and length equal to the number of rows
        #as determined by the function shape[0]
        row_effects = np.zeros(shape=self.tbl.shape[0])
        col_effects = np.zeros(shape=self.tbl.shape[1])

        for i in range(max_iterations):
            if method == 'median':
                #note that np.median's second arg specifies along which axis to perform
                #the median calculation, with axis = 1 being rows, and axis = 0 being columns
                row_medians = np.median(self.tbl, 1)
                #the next line keeps a running total of the row effects that have been
                #subtracted out during the iterative polishing procedure
                row_effects += row_medians
                median_row_effects = np.median(row_effects)
            elif method == 'average':
                row_medians = np.average(self.tbl, 1) 
                row_effects += row_medians
                median_row_effects = np.average(row_effects)
            
            #not sure what the grand_effect variable is keeping track of; it is
            #returned but never used in later calculations
            grand_effect += median_row_effects
            
            #the following line I have commented out, don't know why it is needed
            #row_effects -= median_row_effects
            
            #the following line reshapes the the row_medians array into a column
            #this reshaping is necessary because each element in row_medians
            #is the median from each row, which needs to be subtracted from each row
            #by reshaping into a column, can simply subtract this column from each
            #column of the data matrix, self.tbl
            #the np.newaxis function adds a new dimension, such that now the array
            #is a matrix with n number of rows and 1 column; hence the array is now a
            #column vector.
            self.tbl -= row_medians[:, np.newaxis]

            if method == 'median':
                col_medians = np.median(self.tbl, 0) 
                col_effects += col_medians
                median_col_effects = np.median(col_effects)
            elif method == 'average':
                col_medians = np.average(self.tbl, 0) 
                col_effects += col_medians
                median_col_effects = np.average(col_effects)

            #note that by default, arrays such as col_medians are formulated as a row vector
            #for use in numerical calculations
            self.tbl -= col_medians
            
            #the following line I have commented out, don't know why it is needed
            #col_effects -= col_medians
            
            grand_effect += median_col_effects

        return grand_effect, col_effects, row_effects , self.tbl, self.tbl_org

In [17]:
#sorts data along a given column; can do the reverse (largest to smallest)
def sort_candidates(candidates_list, index_to_rank, flag_reverse_rank):
    if len(candidates_list) == 0:
        return candidates_list
    else:
        data_to_rank = candidates_list[:, index_to_rank]

        if flag_reverse_rank == 1:
            data_to_rank = [-x for x in data_to_rank]

        ranks = scipy.stats.rankdata(data_to_rank)

        ranks = np.asarray(ranks)
        ranks = ranks[:, np.newaxis]

        candidates_list = np.concatenate((ranks, candidates_list), axis = 1)

        candidates_list = np.ndarray.tolist(candidates_list)

        candidates_list.sort()
        candidates_list = np.asarray(candidates_list)
        return(candidates_list)

In [18]:
#static variables

#number of plates, rows, columns
PLATES = 14
PLATE_ROWS = 16
PLATE_COLS = 24

#number of replicates
NUM_REPL = 2

#number of experimental and control groups
NUM_GROUPS = 1

#the actual data is listed in the following rows and columns; the rest are
#data table label strings
FIRSTROW = 4
LASTROW = 388
FIRSTCOLUMN = 1
LASTCOLUMN = 15

#chose 10 nearest neighbors because in a given screen not likely has more than 10 or so hits
#and so only about 10 or so neighboring hits would be informative for imputing a putative
#hit that is missing a value.  for putative non-hits, there are many neighbors with similar
#values and so 10 is still a good number
K_NEAREST_NEIGHBORS = 10

#number of iterations to run Tukey's two median polish
NUM_POLISH_ITER = 10

#1.4826 is scaling constant to make 1 MAD comparable in magnitude to 1 SD
#see (chung, strulovici et al 2007)
MAD_SCALING_CONST = 1.4826

#n MAD away roughly corresponds to false positive rate at n SD away, under normal distribution
THRESHOLD = 3

#significance level for hit detection by method with random variances model
SIGMA = 0.001


In [19]:
#open raw data file
data_file = open('/Users/markfang/Dropbox/UCSD Grad work/RNA-Yeo Lab/Data - analyses/SG project/screening/granule area to nuclear area analyses of large screens/150610 cvb smnpc spectrum screen rewashed/before reDAPI.csv', 'rU')
data_table = np.genfromtxt(data_file, delimiter=',')

In [20]:
#perform missing data imputation on a screen-by-screen basis,
#where a screen is considered to be all screens done in the same day/batch
#of cells

"""KNN (k nearest neighbors) missing data imputation using Euclidean distance to select k
nearest neighbors and using weight averages for the estimation of the missing value.
Missing values are imputed starting with the first column and going down each row in the
first column, then repeating with the next column to the right in the data table."""

data_table = data_table[FIRSTROW:LASTROW, FIRSTCOLUMN:LASTCOLUMN]
original_data_table = data_table

original_numrows = data_table.shape[0]
original_numcols = data_table.shape[1]

data_table = np.reshape(np.ravel(data_table, order = 'F'), (original_numrows * original_numcols / NUM_REPL, NUM_REPL), order = 'F')

numrows = data_table.shape[0]
numcols = data_table.shape[1]

for j in range(numcols):
    for i in range(numrows):
        #find missing values by checking for 'nan' value in ndarray
        if np.isnan(data_table[i][j]):
            #for determining Euclidean distances from the small molecule compound which is
            #missing value in 1 replicate to all the other vectors containing data for
            #all the other small molecule compounds,
            #we ignore the values in the replicate in which there is the missing value
            other_vectors = np.delete(data_table, j, axis = 1)
            
            #initialize the small molecule compound for which
            #there is a missing value for 1 replicate; this vector
            #will be used as the basis to compare Euclidean distance to the other_vectors
            vector_missingval = other_vectors[i:(i + 1), 0:numcols]
            
            #initialize a vector containing the data in the same repl in which there is
            #the missing value; the missing value will be imputed by a weighted average
            #of the other data in this replicate
            estimating_vals = data_table[0:numrows, j:(j + 1)]
            
            other_vectors = np.delete(other_vectors, i, axis = 0)
            estimating_vals = np.delete(estimating_vals, i, axis = 0)
            
            nan_estimator_flag = 1
            
            #remove other rows in which the estimating value is also missing
            while(nan_estimator_flag == 1):
                nan_estimator_flag = 0
                for x in range(len(estimating_vals)):
                    if (np.isnan(estimating_vals[x])):
                        nan_estimator_flag = 1
                        row_with_nan = x
                
                if(nan_estimator_flag == 1):
                    other_vectors = np.delete(other_vectors, row_with_nan, axis = 0)
                    estimating_vals = np.delete(estimating_vals, row_with_nan, axis = 0)
            
            #the following vector will store the Euclidean distances
            euclid_dist = np.zeros((other_vectors.shape[0], 1))

            for k in range(other_vectors.shape[0]):
                flag_nanvector = 0  
                
                for l in range(other_vectors.shape[1]):
                    if (np.isnan(other_vectors[k][l]) == False) and (np.isnan(vector_missingval[0][l]) == False):
                        #calculate Euclidean distance as defined in Troyanskaya et al (2001)
                        euclid_dist[k][0] += (other_vectors[k][l] - vector_missingval[0][l]) ** 2
                        
                        flag_nanvector = 1
                
                if flag_nanvector == 0:
                    #nans are always ranked last
                    euclid_dist[k][0] = data_table[i][j]
            
            #the similarity score calculated below and as defined in Troyanskaya et al (2001)
            #will serve as the weights for imputing the missing value based on weighted
            #average of the remaining - observed - data in the replicate missing the value
            similarity = 1 / euclid_dist
            
            #rank the other small molecule compounds in terms of their Euclidean distance
            #from the small molecule compound with the missing value.  select the k nearest
            #neighbors in terms of Euclidean distance
            simil_rank = scipy.stats.rankdata(euclid_dist, method = "ordinal")
            
            sum_simil = 0
            weighted_avg = 0
            for m in range(other_vectors.shape[0]):
                if simil_rank[m] <= K_NEAREST_NEIGHBORS:
                    sum_simil += similarity[m]
                    weighted_avg += similarity[m] * estimating_vals[m]
            data_table[i][j] = weighted_avg/sum_simil

#reform the original data table, now with the missing values filled in
data_table = np.reshape(np.ravel(data_table, order = 'F'), (original_numrows, original_numcols), order = 'F')
filled_in_data_table = data_table

print(filled_in_data_table)

[[ 0.11481  0.19386  0.12948 ...,  0.17764  0.19573  0.17962]
 [ 0.09561  0.21453  0.1918  ...,  0.18056  0.17152  0.16565]
 [ 0.07482  0.18663  0.17331 ...,  0.16826  0.16843  0.16642]
 ..., 
 [ 0.07334  0.14901  0.21454 ...,  0.19532  0.22694  0.21764]
 [ 0.08439  0.13008  0.19741 ...,  0.20114  0.21603  0.19482]
 [ 0.08699  0.13818  0.19788 ...,  0.183    0.1975   0.18676]]


In [21]:
#perform polish and calculate b scores on a plate-by-plate basis
#where each plate's data is listed as a separate column in the original csv

if __name__ == "__main__":

    #makes output more legible for debugging
    np.set_printoptions(precision = 5, suppress = True)

    num_rows = data_table.shape[0]
    num_cols = data_table.shape[1]
    
    #the data table needs to be reshaped into plates with rows and columns
    #this way I can run the polish on each of the plates.  the next few lines
    #accomplish this reshaping
    #note that by entering a 1-tuple for size, reshape will make a 1D array; no
    #need to actually specify that the array has 1 row or 1 col, as functionally
    #np will treat that as a 2D array with 1 row or 1 col
    data_table = np.reshape(data_table, (num_rows * num_cols), order = 'F')

    #reshape into data organized by plate, row, col
    data_table = np.reshape(data_table, (PLATES, PLATE_ROWS, PLATE_COLS), order = 'C')

    bscores = []
    bscores_printed = np.zeros((PLATE_ROWS * PLATE_COLS, PLATES))
    resid_printed = np.zeros((PLATE_ROWS * PLATE_COLS, PLATES))
    
    #perform polish and calculate b scores on a plate-by-plate basis
    for i in xrange(0, PLATES):
    
        #iterate over each plate and perform the median polish
        arr = data_table[i, :, :]

        tbl_avg = np.average(arr)
        #subtract out the average for each plate, thus normalizing out plate effects
        arr -= tbl_avg
        #pass each plate's data matrix into the MedianPolish object
        mp = MedianPolish(arr)

        #first argument indicates number of iterations to be run
        #ce is an ndarray storing the column effects after n iterations of polishing
        #re is an ndarray storing the row effects after n iterations of polishing
        #resid is the data table that has been polished to remove 
        ge, ce, re, resid, tbl_org =  mp.median_polish(NUM_POLISH_ITER, "median") 

        re_reshape = re[:, np.newaxis]
        
        #the tbl_org returned by mmp.median_polish has had the tbl_avg subtracted,
        #so to get the initial data table back need to add the tbl_avg back
        tbl_org += tbl_avg

        #the next few lines compute the median absolute deviation
        #MAD = median(|x - median(x)|)
        tbl_resid_minusmedians = resid - np.median(resid)
        median_absdev = np.median(np.absolute(tbl_resid_minusmedians))
        
        #find the b scores of the plate
        tbl_bscore = resid / median_absdev
        
        tbl_bscore_toprint = np.reshape(np.ravel(tbl_bscore, order = 'C'), (PLATE_ROWS * PLATE_COLS), order = 'F')
        bscores_printed[:, i] = tbl_bscore_toprint
        
        resid_toprint = np.reshape(np.ravel(resid, order = 'C'), (PLATE_ROWS * PLATE_COLS), order = 'F')
        resid_printed[:, i] = resid_toprint
        
        #convert tbl_bscore from ndarray to simple list
        tbl_bscore = tbl_bscore.tolist()
        
        #collect the b scores of each plate into the array bscores
        bscores.append(tbl_bscore)
    
    bscores = np.asarray(bscores)
    
    print(bscores_printed)

[[ 2.04251 -0.6578  -4.10163 ...,  2.4048   4.35924  4.03549]
 [-2.93128  2.68495  1.20778 ...,  1.84737  1.2637   0.48307]
 [-5.17089 -1.21856 -0.4671  ...,  0.85046 -0.00563 -0.00873]
 ..., 
 [-2.80828 -3.82109 -0.06561 ..., -1.49032  0.23867 -0.86566]
 [-0.94212 -7.24117 -2.33834 ..., -1.74661 -1.15266 -3.83698]
 [ 4.31385 -0.11927  1.04225 ...,  1.17598  1.33829  0.92417]]


In [22]:
#select hits based on median and median absolute deviation on a screen-by-screen
#basis, where a screen is considered to be all screens done in the same day/batch
#of cells
INDEX_OFFSET = 1

bscores_median = np.median(bscores)

bscores_copy = bscores
#subtract the overall median; this is later used to calculate the MAD
for i in range(0, PLATES):
    for j in range(0, PLATE_ROWS):
        for k in range(0, PLATE_COLS):
            bscores_copy[i][j][k] -= bscores_median

mad = MAD_SCALING_CONST * np.median(np.absolute(bscores_copy))
    
upper_threshold = bscores_median + THRESHOLD * mad
lower_threshold = bscores_median - THRESHOLD * mad

hits_mad = []
sg_enhancers_mad = []
    
unique_plates = PLATES / NUM_REPL

for i in range(0, unique_plates):
    for j in range(0, PLATE_ROWS):
        for k in range(0, PLATE_COLS):
            #find median of replicates, then check if it is +/- 3 MAD
            arr_to_find_median = []
            
            for l in range(0, NUM_REPL):
                arr_to_find_median.append(bscores[i + (l * unique_plates)][j][k])
            
            median_ofrepl = np.median(arr_to_find_median)
            
            if median_ofrepl < lower_threshold:
                hits_mad.append(median_ofrepl)
                
                #collect plate, row, column coordinates of hits
                #indexed from 1 rather than 0
                hits_mad.append(i + INDEX_OFFSET)
                hits_mad.append(j + INDEX_OFFSET)
                hits_mad.append(k + INDEX_OFFSET)
                
                hits_mad = hits_mad + arr_to_find_median
                
            if median_ofrepl > upper_threshold:
                sg_enhancers_mad.append(median_ofrepl)

                sg_enhancers_mad.append(i + INDEX_OFFSET)
                sg_enhancers_mad.append(j + INDEX_OFFSET)
                sg_enhancers_mad.append(k + INDEX_OFFSET)
                
                sg_enhancers_mad = sg_enhancers_mad + arr_to_find_median


MAD_RESULTS_NUM_COLS = 1 + 3 + NUM_REPL

len_hits_mad = len(hits_mad)
hits_mad = np.asarray(hits_mad)

#reshape into median, ordered triples giving the plate, row, and column coordinates for hits
#as well as raw values from replicates
hits_mad = np.reshape(hits_mad, (len_hits_mad / MAD_RESULTS_NUM_COLS, MAD_RESULTS_NUM_COLS), order = 'C')

#rank and sort hits by the distance of the median of the replicates from the median
#of the entire screen
#where the entire screen is defined as all compounds tested on same day
hits_mad = sort_candidates(hits_mad, 0, 0)
print(hits_mad)


len_sg_enhancers_mad = len(sg_enhancers_mad)
sg_enhancers_mad = np.asarray(sg_enhancers_mad)
sg_enhancers_mad = np.reshape(sg_enhancers_mad, (len_sg_enhancers_mad / MAD_RESULTS_NUM_COLS, MAD_RESULTS_NUM_COLS), order = 'C')
sg_enhancers_mad = sort_candidates(sg_enhancers_mad, 0, 1)
print(sg_enhancers_mad)

[[  1.      -17.98465   1.       10.       11.      -11.72439 -24.24492]
 [  2.      -17.16858   3.        6.       16.      -14.5438  -19.79335]
 [  3.      -15.69228   1.        6.       23.        3.18074 -34.56531]
 [  4.      -15.53996   6.       11.       11.       -5.45194 -25.62798]
 [  5.      -14.81937   1.        5.       23.        4.32083 -33.95957]
 [  6.      -13.34303   5.        3.       24.      -26.69453   0.00847]
 [  7.      -13.00774   1.       15.       13.        1.15417 -27.16966]
 [  8.      -12.82784   1.        2.       17.      -10.56684 -15.08885]
 [  9.      -12.77983   6.        7.       19.        0.39715 -25.95681]
 [ 10.      -12.7661    4.        3.       21.      -25.53196  -0.00024]
 [ 11.      -12.5686    6.       15.        9.        0.35441 -25.49161]
 [ 12.      -11.93021   6.        8.       19.        1.98215 -25.84257]
 [ 13.      -11.58548   3.       12.        7.      -17.54878  -5.62218]
 [ 14.      -11.4654    1.       12.        5.     

In [23]:
#fit variances to random variance model (inverse gamma distributed) on a screen-by-screen
#basis, where a screen is considered to be all screens done in the same day/batch
#of cells

variances = []
averages = []

for i in range(0, unique_plates):
    for j in range (0, PLATE_ROWS):
        for k in range (0, PLATE_COLS):
            sum_of_repl = 0
            sum_of_var = 0
            
            for l in range(0, NUM_REPL):
                sum_of_repl += (bscores[i + (l * unique_plates)][j][k])
            
            sample_avg = sum_of_repl / NUM_REPL

            for m in range(0, NUM_REPL):
                sum_of_var += ((bscores[i + (m * unique_plates)][j][k]) - sample_avg) ** 2

            sample_variance = sum_of_var / (NUM_REPL - 1)
            
            variances.append(sample_variance)
            averages.append(sample_avg)

#according to paper wright and simon 2003, the sample variances multipled by two constants
#a and b follow an F distribution with parameters (n-k) and 2a, where n is number of
#replicates, k is number of group (experimental, control, etc)
#in my data, I have duplicates and 1 group, so n = 2, k = 1.
param = scipy.stats.f.fit(variances, f0 = NUM_REPL - NUM_GROUPS)
print(param)

#after fitting we want to find the value of a and b, since these are the parameters for
#the putative inverse gamma distribution that is the true distribution of the variances
#of the small molecule screen.  finding a and b will help us specify the inverse
#gamma distribution, which will improve the power of our t tests (wright and simon 2003)
#find parameter a: since the fitted distribution has parameters (n-k) and 2a, we can
#take the second parameter and divide by 2 to get a
invgammaparam_a = param[1] / 2

#we fit an F distribution to our variances, and we see that the scaling s is stored in the
#fourth parameter.  a*b*variances fits to an F distribution with area under the curve = 1
#since F is a probability distribution (scaling = 1)
#thus when we simply fit our variances to an F distribution,
#we may get a scaling s =/= 1 (area under the curve not equal 1)
#since multiplying variates by a constant changes the scaling of the fitted F distribution
#we can figure out what a*b is by knowing that multiplying the variances
#by a*b brings the scaling up to 1; hence a*b equals the multiplicative inverse
#of the current scaling.  from here we can find b because we already have a
invgammaparam_b = (1 / param[3]) / invgammaparam_a
print(invgammaparam_a)
print(invgammaparam_b)

print(variances[0])
new_variances = [(invgammaparam_a * invgammaparam_b * x) for x in variances]
print(new_variances[0])
param = scipy.stats.f.fit(new_variances, f0 = NUM_REPL - NUM_GROUPS)
print(param)
print(variances[0])

(1, 2.8247942203958747, 1.1915299915352198e-06, 1.814355013440947)
1.4123971102
0.390230230621
0.00380558484244
0.00209748633219
(1, 2.3097852857103325, 6.5672372975973879e-07, 0.78790846533795844)
0.00380558484244


In [24]:
#select hits based on t tests (under random variance model), on a screen-by-screen
#basis, where a screen is considered to be all screens done in the same day/batch
#of cells

#convert lists to numpy objects ndarrays to be able to easily perform math operations
variances = np.asarray(variances)
averages = np.asarray(averages)

#variances that have been fitted to the inverse gamma distribution
rvm_variances = ((NUM_REPL - 1) * variances + 2 * invgammaparam_a * (1 / (invgammaparam_a * invgammaparam_b))) / ((NUM_REPL - 1) + 2 * invgammaparam_a)          

#calculate std dev from the variances that have been fitted to the inverse gamma distribution
denominator = np.sqrt(rvm_variances / NUM_REPL)

#calculate t statistic for each compound, using the rvm_variances
t_stats = (averages - 0) / denominator

len_t_stats = t_stats.shape[0]

p_val = []
df = NUM_REPL - 1 + (2 * invgammaparam_a)

#p values for 2 tailed t tests
for i in range(0, len_t_stats):
    if t_stats[i] <= 0:
        prob = scipy.stats.t.cdf(t_stats[i], df)
        prob *= 2
        p_val.append(prob)
    else:
        prob = scipy.stats.t.sf(t_stats[i], df)
        prob *= 2
        p_val.append(prob)

p_val = np.asarray(p_val)
p_val = p_val[:, np.newaxis]

coordinates = []

#list out the plate, row, and col coordinates to be concatenated with the p values
#this helps keep track of where each p value came from in the physical location
#on the plates after the p values are sorted in order to do FDR controlling
#such as benjamini hochberg
#these are indexed from 1 not 0 for ease of interpretation (i.e. plate 1 rather than plate 0)
for i in range(0, unique_plates):
    for j in range(0, PLATE_ROWS):
        for k in range(0, PLATE_COLS):
            coordinates.append(i + INDEX_OFFSET)
            coordinates.append(j + INDEX_OFFSET)
            coordinates.append(k + INDEX_OFFSET)

coordinates = np.asarray(coordinates)
len_coord = len(coordinates)
coordinates = np.reshape(coordinates, (len_coord / 3, 3), order = 'C')

p_val_coord = np.concatenate((p_val, coordinates), axis = 1)
p_val_coord = np.ndarray.tolist(p_val_coord)

p_val_coord.sort()
print(p_val_coord)

print(t_stats)
print(averages)

[[0.00010608235502017774, 1.0, 1.0, 8.0], [0.0002751496738095974, 4.0, 11.0, 20.0], [0.0002997435801331773, 1.0, 11.0, 21.0], [0.0004563226982692392, 1.0, 14.0, 18.0], [0.0005056018694296892, 3.0, 6.0, 16.0], [0.000605115284312977, 2.0, 8.0, 18.0], [0.0008466833885226506, 3.0, 4.0, 21.0], [0.0009395698447141293, 3.0, 15.0, 4.0], [0.0010122564207729397, 1.0, 2.0, 17.0], [0.001385688845535345, 1.0, 12.0, 5.0], [0.001396947191140562, 3.0, 12.0, 18.0], [0.002009177093740718, 1.0, 11.0, 13.0], [0.002336401883631327, 3.0, 5.0, 3.0], [0.002363504410057932, 3.0, 3.0, 18.0], [0.0029132216814361767, 6.0, 13.0, 3.0], [0.0035124777584752756, 2.0, 7.0, 17.0], [0.003643162805308993, 1.0, 1.0, 3.0], [0.0040394090455007576, 4.0, 1.0, 18.0], [0.004125821505099942, 2.0, 9.0, 22.0], [0.004598285401512845, 2.0, 9.0, 12.0], [0.005064100767128696, 2.0, 16.0, 17.0], [0.005738093339202955, 1.0, 13.0, 6.0], [0.005952195130887907, 1.0, 10.0, 7.0], [0.006195862833165733, 7.0, 9.0, 22.0], [0.006287354172262562, 1

In [25]:
#select hits based on t tests (under random variance model), on a screen-by-screen
#basis, where a screen is considered to be all screens done in the same day/batch
#of cells

#ignoring multiple hypothesis testing and therefore not controlling for FDR
hits_t_test = []
sg_enhancers_t_test = []

len_p_val_coord = len(p_val_coord)

for i in range(0, len_p_val_coord):
    if p_val_coord[i][0] < SIGMA:
        plate_position = p_val_coord[i][1] - 1
        row_position = p_val_coord[i][2] - 1
        col_position = p_val_coord[i][3] - 1
        
        plate_buffer = (plate_position) * PLATE_ROWS * PLATE_COLS
        row_buffer = (row_position) * PLATE_COLS
        col_buffer = col_position
        
        arr_to_find_avg = []
        sum_of_repl = 0
        
        for j in range(0, NUM_REPL):
            arr_to_find_avg.append(bscores[plate_position + (j * unique_plates)][row_position][col_position])
            sum_of_repl += (bscores[plate_position + (j * unique_plates)][row_position][col_position])
            
        sample_avg = [sum_of_repl / NUM_REPL]
        
        compound_t_stat = t_stats[plate_buffer + row_buffer + col_buffer]
    
        compound_entry = sample_avg + p_val_coord[i] + arr_to_find_avg

        if compound_t_stat <= 0:
            hits_t_test.append(compound_entry)
        else:
            sg_enhancers_t_test.append(compound_entry)

hits_t_test = np.asarray(hits_t_test)
sg_enhancers_t_test = np.asarray(sg_enhancers_t_test)

hits_t_test = sort_candidates(hits_t_test, 0, 0)
sg_enhancers_t_test = sort_candidates(sg_enhancers_t_test, 0, 1)
            
print(len(hits_t_test))        
print(hits_t_test)
print(len(sg_enhancers_t_test))
print(sg_enhancers_t_test)

6
[[  1.      -17.16858   0.00051   3.        6.       16.      -14.5438
  -19.79335]
 [  2.      -10.99846   0.00028   4.       11.       20.      -11.48749
  -10.50943]
 [  3.      -10.40463   0.0003    1.       11.       21.      -10.1559
  -10.65337]
 [  4.      -10.39365   0.00085   3.        4.       21.      -11.8117
   -8.9756 ]
 [  5.       -9.19095   0.00046   1.       14.       18.       -9.15565
   -9.22626]
 [  6.       -7.727     0.00094   3.       15.        4.       -7.39269
   -8.06131]]
2
[[  1.       16.01248   0.00011   1.        1.        8.       15.00339
   17.02157]
 [  2.        8.5614    0.00061   2.        8.       18.        8.72297
    8.39982]]


In [26]:
#benjamini hochberg method to control FDR
FDR = 0.05
k = 1.0
i = 0
m = unique_plates * PLATE_ROWS * PLATE_COLS

hits_fdr = []

print(k / m * FDR)

while p_val_coord[i][0] < (k / m * FDR):
    hit = p_val_coord[i]
    hits_fdr.append(hit)
    i += 1
    k += 1.0
    
print(hits_fdr)

1.86011904762e-05
[]


In [27]:
#write out results to text files
with open ('150706 sg results/cvb smnpc spectrum reDAPI analyses.csv','w') as analysis_outputfile:
    writer = csv.writer(analysis_outputfile, delimiter = ',')

    writer.writerows([['original data']])
    writer.writerows(original_data_table)
    writer.writerows([[' ']])
    
    writer.writerows([['KNN missing value imputed data']])
    writer.writerows(filled_in_data_table)
    writer.writerows([[' ']])

    writer.writerows([['after Tukey two median polish']])
    writer.writerows(resid_printed)
    writer.writerows([[' ']])
    
    writer.writerows([['b scores']])
    writer.writerows(bscores_printed)
    writer.writerows([[' ']])    
    
    writer.writerows([['hits using MAD']])
    writer.writerows(hits_mad)
    writer.writerows([[' ']])
    
    writer.writerows([['hits using random variates model']])
    writer.writerows(hits_t_test)
    writer.writerows([[' ']])

    writer.writerows([['sg enhancers using MAD']])
    writer.writerows(sg_enhancers_mad)
    writer.writerows([[' ']])
    
    writer.writerows([['sg enhancers using random variates model']])
    writer.writerows(sg_enhancers_t_test)
    writer.writerows([[' ']])

In [28]:
#write out analysis parameters to text files
with open ('150706 sg results/cvb smnpc spectrum reDAPI analyses parameters.csv','w') as analysis_param_outputfile:
    writer = csv.writer(analysis_param_outputfile, delimiter = ',')
    writer.writerows([['raw data parameters']])
    writer.writerows([['num of plates:', ' ', ' ', '%i' % PLATES]])
    writer.writerows([['num of rows:', ' ', ' ' , '%i' % PLATE_ROWS]])
    writer.writerows([['num of cols:', ' ', ' ' , '%i' % PLATE_COLS]])
    writer.writerows([['num of replicates:', ' ', ' ' , '%i' % NUM_REPL]])
    writer.writerows([['num of contrl and exp groups:', ' ', ' ' , '%i' % NUM_GROUPS]])
    writer.writerows([[' ']])
    writer.writerows([['analysis parameters']])
    writer.writerows([['num of k nearest neighbors:', ' ', ' ' , '%i' % K_NEAREST_NEIGHBORS]])
    writer.writerows([['num of polish iterations:', ' ', ' ' , '%i' % NUM_POLISH_ITER]])
    writer.writerows([['median abs dev scaling is:', ' ', ' ' , '%f' % MAD_SCALING_CONST]])
    writer.writerows([['median abs dev threshold during hit selection:', ' ', ' ' , '%i' % THRESHOLD]])
    writer.writerows([['significance level during hit selection:', ' ', ' ' , '%f' % SIGMA]])